<a href="https://colab.research.google.com/github/asnoldy02-cell/sds510/blob/main/Module_4_Basic.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [59]:
!pip install beautifulsoup4

import requests
from bs4 import BeautifulSoup
import csv
import time
import random
from datetime import datetime
from google.colab import files
import os

In [61]:
SECTION_URLS = {
    "Science": "https://news.asu.edu/interests/science",
    "University News": "https://news.asu.edu/saf/university-news",
    "Technology": "https://news.asu.edu/interests/technology",
    "Research": "https://news.asu.edu/interests/research"

}

KEYWORDS = ["medicine", "breakthrough", "upcoming", "research", "drug", "explosive", "formula"]
OUTPUT_FILE = "asu_keyword_hits.csv"
SEEN_FILE = "seen_articles.txt"
HEADERS = {"User-Agent": "Mozilla/5.0 (compatible; OSINT-Monitor/1.0)"}


In [62]:
def load_seen_articles():
    if os.path.exists(SEEN_FILE):
        with open(SEEN_FILE, "r") as f:
            return set(line.strip() for line in f)
    return set()

def save_seen_articles(seen_set):
    with open(SEEN_FILE, "w") as f:
        for url in seen_set:
            f.write(url + "\n")

def get_article_links(section_url):
    """Fetch article URLs from a section page."""
    resp = requests.get(section_url, headers=HEADERS, timeout=10)
    soup = BeautifulSoup(resp.text, "html.parser")
    links = set()

In [65]:
def get_article_links(section_url):
    """Fetch article URLs from a section page."""
    resp = requests.get(section_url, headers=HEADERS, timeout=10)
    soup = BeautifulSoup(resp.text, "html.parser")
    links = set()

    for a_tag in soup.find_all("a", href=True):
        href = a_tag["href"]
        if href.startswith("/") and len(href.split("/")[1].split("-")[0]) == 8:  # YYYYMMDD pattern
            full_url = "https://news.asu.edu" + href
            links.add(full_url)
    return list(links)

def check_article_for_keywords(url):
    """Visit article and check for keywords."""
    try:
        time.sleep(random.uniform(2, 5))
        resp = requests.get(url, headers=HEADERS, timeout=10)
        soup = BeautifulSoup(resp.text, "html.parser")
        text = soup.get_text(separator=" ").lower()
        title_tag = soup.find("h1")
        title = title_tag.get_text(strip=True) if title_tag else "No Title"
        found_keywords = [kw for kw in KEYWORDS if kw.lower() in text]
        return found_keywords, title
    except Exception as e:
        print(f"Error fetching {url}: {e}")
        return [], "Error"

def log_results(timestamp, section, keyword, url, title):
    with open(OUTPUT_FILE, "a", newline="", encoding="utf-8") as f:
        writer = csv.writer(f)
        writer.writerow([timestamp, section, keyword, url, title])

In [66]:
if not os.path.exists(OUTPUT_FILE):
    with open(OUTPUT_FILE, "w", newline="", encoding="utf-8") as f:
        writer = csv.writer(f)
        writer.writerow(["timestamp", "section", "keyword", "url", "title"])

seen_articles = load_seen_articles()

print("Starting OSINT monitoring across all ASU News sections...")

for section, url in SECTION_URLS.items():
    print(f"\nScraping section: {section}")
    article_links = get_article_links(url)
    print(f"Found {len(article_links)} articles in {section}.")

    for link in article_links:
        if link in seen_articles:
            continue
        found_keywords, title = check_article_for_keywords(link)
        timestamp = datetime.now().isoformat()
        for kw in found_keywords:
            print(f"🔎 Keyword '{kw}' found in {link} ({section})")
            log_results(timestamp, section, kw, link, title)
        seen_articles.add(link)

Starting OSINT monitoring across all ASU News sections...

Scraping section: Science
Found 1 articles in Science.
🔎 Keyword 'medicine' found in https://news.asu.edu/20251105-science-and-technology-thousands-mammals-and-counting-hidden-biodiversity-growing (Science)
🔎 Keyword 'research' found in https://news.asu.edu/20251105-science-and-technology-thousands-mammals-and-counting-hidden-biodiversity-growing (Science)

Scraping section: University News
Found 1 articles in University News.
🔎 Keyword 'medicine' found in https://news.asu.edu/20251107-university-news-asu-state-arizona-launch-initiative-power-public-workforce (University News)

Scraping section: Technology
Found 0 articles in Technology.

Scraping section: Research
Found 1 articles in Research.
🔎 Keyword 'medicine' found in https://news.asu.edu/20251107-health-and-medicine-asu-researcher-importance-considering-sex-differences-health-studies (Research)
🔎 Keyword 'research' found in https://news.asu.edu/20251107-health-and-medici

In [67]:
save_seen_articles(seen_articles)

print("\nMonitoring complete. Results saved to:", OUTPUT_FILE)
files.download(OUTPUT_FILE)


Monitoring complete. Results saved to: asu_keyword_hits.csv


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>